In [1]:
import re
import numpy as np
from itertools import product
from functools import cache
from tqdm import tqdm 

In [2]:
with open("./data_inputs/day19_input.txt") as f:
    input_raw = f.read()

# blueprint = {id: [robot_type: [ore, clay, obsidian]]}
blueprints = {}
for i, l in enumerate(input_raw.split("\n")):
    blueprints[i+1] = ((int(re.findall(r"costs \d+", l.split(".")[0])[0][6:]), 0, 0),
                        (int(re.findall(r"costs \d+", l.split(".")[1])[0][6:]), 0, 0),
                        (int(re.findall(r"costs \d+", l.split(".")[2])[0][6:]), int(re.findall(r"and \d+", l.split(".")[2])[0][4:]), 0),
                        (int(re.findall(r"costs \d+", l.split(".")[3])[0][6:]), 0, int(re.findall(r"and \d+", l.split(".")[3])[0][4:])))

In [13]:
blueprints[1]

((3, 0, 0), (4, 0, 0), (2, 20, 0), (4, 0, 7))

In [4]:
list(product(range(4),range(1),range(2)))

[(0, 0, 0),
 (0, 0, 1),
 (1, 0, 0),
 (1, 0, 1),
 (2, 0, 0),
 (2, 0, 1),
 (3, 0, 0),
 (3, 0, 1)]

In [5]:
list(product(*list(map(range, np.array([1,2,4])))))

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3)]

In [6]:
np.sum(np.array([1,2,3,0]).reshape(-1, 1) * np.array([[1,1,1], [2,3,4], [5,6,7], [6,7,7]]), axis=0)

array([20, 25, 30])

In [7]:
np.array((1,2,4,4)).reshape(4, 1).shape

(4, 1)

In [ ]:
# [ore_r, clay_r, obsidian_r, geode_r]

@cache
def robots_max_geode(minute, blueprint, robots, materials):
    materials = np.array(materials)
    robots = np.array(robots)
    blueprint_arr = np.array(blueprint)

    if minute >= 24:
        print(minute, robots, materials)
        materials += robots
        return materials[-1]

    possible_new_robots = np.array([0, 0, 0, 0], dtype=np.uint8)
    for i, robot_materials in reversed(list(enumerate(blueprint_arr))):
        while np.all(robot_materials <= (materials - np.array([*np.sum(possible_new_robots.reshape(4, 1) * blueprint_arr, axis=0), 0]))[:3]):
            possible_new_robots[i] += 1

    max_geode = -1
    new_robots = np.array([0, 0, 0, 0], dtype=np.uint8)
    for _new_robots in product(*list(map(range, possible_new_robots+1))):
        geode = robots_max_geode(minute+1, blueprint, tuple(robots + _new_robots), tuple(materials + robots - np.array([*np.sum((np.array(_new_robots).reshape(4, 1))*blueprint_arr, axis=0), 0])))
        if max_geode < geode:
            max_geode = geode
            new_robots = _new_robots
    
    # TODO: discount the amount of material spent by the possible new robots and for the finally created new robots
    materials += robots
    robots += new_robots
    
    return materials[-1]


quality_level = 0

for id in tqdm(blueprints):
    robots = np.array([1, 0, 0, 0], dtype=np.uint8)
    materials = np.array([0, 0, 0, 0], dtype=np.uint8)  
    blueprint = blueprints[id]
    
    max_geode = robots_max_geode(1, blueprint, tuple(robots), tuple(materials))

    quality_level += id * materials[-1]
    #print(robots, materials)

print("Result 1:", quality_level)


In [47]:
for id in blueprints:
    print(id)
    for i, robot in reversed(list(enumerate(blueprints[id].keys()))):
        print(i, robot)
    break

1


TypeError: 'enumerate' object is not reversible